In [1]:
from  selenium import webdriver 
from bs4 import BeautifulSoup as bs
import re 
import string
from time import  sleep
from selenium.common.exceptions import TimeoutException
import pickle
from collections import deque
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as  np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
with open("./vilas_state.pkl",'rb') as file:
        vilas,stack2,link2=pickle.load(file)
df_vilas=pd.DataFrame(vilas)
df_vilas.rename(columns={'data': 'date'},inplace=True, errors='raise')
df_vilas.drop(['vlaues'],axis=1,inplace=True)
df_vilas.drop(df_vilas.loc[[type(v)!=str for v in df_vilas['date']]].index,inplace=True)
df_vilas['cat']='Sell'
len(vilas)

21737

In [3]:
with open("./vlias-for-rent.pkl",'rb') as file:
        vilas_rent,stack2,link2=pickle.load(file)
df_vilas_rent=pd.DataFrame(vilas_rent)
df_vilas_rent.rename(columns={'data': 'date'},inplace=True, errors='raise')
df_vilas_rent.drop([i  for i in list(df_vilas_rent.columns) if i not in list(df_vilas.columns)],inplace=True,axis=1)
df_vilas_rent['cat']='Rent'
len(vilas_rent)

5142

In [4]:
with open("./apartmant.pkl",'rb') as file:
        apartmant,stack2,link2=pickle.load(file)
df_apartmant=pd.DataFrame(apartmant)
df_apartmant.rename(columns={'data': 'date'},inplace=True, errors='raise')
df_apartmant['cat']='Sell'

len(apartmant)

20317

In [5]:
with open("./apertment-rent.pkl",'rb') as file:
        apertment_rent,stack2,link2=pickle.load(file)
df_apertment_rent=pd.DataFrame(apertment_rent)
df_apertment_rent.rename(columns={'data': 'date'},inplace=True, errors='raise')
df_apertment_rent['cat']='Rent'
len(apertment_rent)

21735

In [6]:
with open("state21.pkl",'rb') as file:
        state21,stack2,link2=pickle.load(file)
df_state21=pd.DataFrame(state21)
len(state21)

6518

In [7]:
df=[df_apertment_rent ,df_apartmant ,df_vilas_rent,df_vilas]

In [8]:
df_apartmant['Level'].value_counts() 

[2]          4189
[3]          3396
[1]          2951
[Ground]     2761
[4]          2011
[5]          1512
[6]           771
[7]           573
[10+]         558
[9]           431
[8]           428
[10]          375
[Highest]     361
Name: Level, dtype: int64

***Cleaning***


In [9]:
df_apertment_rent[[i for i in list(df[1].columns) if i not in list(df[0].columns)]]=np.nan
df_vilas_rent[[i for i in list(df[1].columns) if i not in list(df[3].columns)]]=np.nan
df_vilas[[i for i in list(df[1].columns) if i not in list(df[3].columns)]]=np.nan

In [10]:
df_apertment_rent['Level'].value_counts()

[2]          4019
[3]          3514
[Ground]     3435
[1]          3202
[4]          2636
[5]          1672
[6]           786
[7]           574
[10+]         481
[8]           469
[9]           440
[10]          375
[Highest]     131
[]              1
Name: Level, dtype: int64

In [11]:
for i in df:
    i['date']=i['date'].apply(lambda x: x.strip())
    i['date']=pd.to_datetime(i['date'], format="%d %B %Y")

In [12]:
z=list(df_vilas.columns)
x=['price','location','date','title','link','viwers','negotiable','cat']
selected=[i for i in z if i not in x ]
selected

['Bedrooms',
 'Bathrooms',
 'Area (m²)',
 'Type',
 'Payment Option',
 'Compound',
 'Delivery Term',
 'Amenities',
 'Furnished',
 'Delivery Date',
 'Down Payment',
 'Level']

In [13]:
for x in df:
    for i in selected:
        print(df_vilas[i].value_counts())
        print("*"*80)

[4]      8826
[3]      4968
[5]      4224
[6]      1798
[7]       618
[10+]     536
[8]       235
[2]       168
[10]      148
[9]       135
[1]        47
Name: Bedrooms, dtype: int64
********************************************************************************
[4]      7550
[3]      6340
[5]      3637
[6]      1537
[2]      1009
[7]       632
[10+]     410
[8]       240
[1]       153
[9]        98
[10]       97
Name: Bathrooms, dtype: int64
********************************************************************************
[300]       548
[400]       504
[600]       484
[500]       469
[260]       401
           ... 
[1,375]       1
[443]         1
[609]         1
[1,301]       1
[388.78]      1
Name: Area (m²), Length: 1078, dtype: int64
********************************************************************************
[Stand, Alone, Villa]    11528
[Town, House]             5770
[Twin, House]             4405
Name: Type, dtype: int64
****************************************************

In [14]:
def xx (z):
    try:
        return ' '.join(str(e) for e in z)
    except Exception as e:
        
        return np.nan
     
for x in df:
    for i in selected:
        x[i]=x[i].apply(xx)

In [15]:
for i in selected:
    print(i)
    print(df_vilas[i].value_counts())
    print("*"*80)

Bedrooms
4      8826
3      4968
5      4224
6      1798
7       618
10+     536
8       235
2       168
10      148
9       135
1        47
Name: Bedrooms, dtype: int64
********************************************************************************
Bathrooms
4      7550
3      6340
5      3637
6      1537
2      1009
7       632
10+     410
8       240
1       153
9        98
10       97
Name: Bathrooms, dtype: int64
********************************************************************************
Area (m²)
300       548
400       504
600       484
500       469
260       401
         ... 
1,375       1
443         1
609         1
1,301       1
388.78      1
Name: Area (m²), Length: 1078, dtype: int64
********************************************************************************
Type
Stand Alone Villa    11528
Town House            5770
Twin House            4405
Name: Type, dtype: int64
********************************************************************************
Payment Option


In [16]:
def x(z):
    if type(i) == str:
        if z.endswith('+'):
            return int(z[:-1])
        else:
            return int(z)
    else:
        return z
for i in df:
    i['Bathrooms']=i['Bathrooms'].apply(x)

In [17]:
def x(z):
    try:
        return int(z)
    except:
        if z=='Ground':
            return 0
        elif z=='Highest':
            return 11
        elif z=='10+':
            return 10
        else:
            print(z)
            return z

for i in df[:2]:
    i['Level']=i['Level'].apply(x)
for i in df[:2]:
    print(i['Level'].value_counts())
    print('*'*80)


2     4019
3     3514
0     3435
1     3202
4     2636
5     1672
10     856
6      786
7      574
8      469
9      440
11     131
         1
Name: Level, dtype: int64
********************************************************************************
2     4189
3     3396
1     2951
0     2761
4     2011
5     1512
10     933
6      771
7      573
9      431
8      428
11     361
Name: Level, dtype: int64
********************************************************************************


In [18]:
def x(z):
    try:
        x=''
        if type(z)==str: 
            for i in z:
                if  i.isdigit():
                    x+=i
            return float(x)
        elif type(z)==list:
            print(z,'list')
            for i in z[0]:
                if i.isdigit():
                    x+=i
            return float(x)
        else:
            return np.nan
    except:
        return np.nan
for  i in df :
    i['price']=         i['price'].apply(x)
    i['Down Payment']=  i['Down Payment'].apply(x)
    i['Area (m²)']=     i['Area (m²)'].apply(x)
    i['viwers'] =       i['viwers'].astype(int)
    i['negotiable'] =   i['negotiable'].astype(int)

In [19]:
df_apertment_rent.loc[df_apertment_rent['Level']=='','Level']=np.nan
df_apartmant.loc[df_apertment_rent['Level']=='','Level']=np.nan


In [20]:

# for i in df:
#     print(df[0].dtypes ==i.dtypes)

x=df[0].dtypes.values !=  df[1].dtypes.values
z=list(df_apartmant.columns)
s=[]
c=[i for i in range(len(x)) if x[i]==True]
for i in c:
    s.append(z[i])
type(df[0][s]['Delivery Date'][0])

numpy.float64

In [21]:
for i in df:
    print(i.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21735 entries, 0 to 21734
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   price           21735 non-null  float64       
 1   location        21735 non-null  object        
 2   date            21735 non-null  datetime64[ns]
 3   title           21735 non-null  object        
 4   link            21735 non-null  object        
 5   viwers          21735 non-null  int32         
 6   negotiable      21735 non-null  int32         
 7   Bedrooms        21735 non-null  object        
 8   Bathrooms       21735 non-null  object        
 9   Area (m²)       21735 non-null  float64       
 10  Furnished       17048 non-null  object        
 11  Level           21734 non-null  object        
 12  Type            21731 non-null  object        
 13  Compound        6361 non-null   object        
 14  Amenities       14587 non-null  object        
 15  Do

In [22]:
for i in df:
    print(i['Down Payment'].value_counts())

5.000000e+03    257
6.000000e+03    215
1.000000e+04    202
3.000000e+03    198
8.000000e+03    192
               ... 
1.550000e+03      1
1.010199e+11      1
5.500000e+01      1
2.500000e+01      1
9.800000e+03      1
Name: Down Payment, Length: 315, dtype: int64
2.000000e+05    124
3.000000e+05    121
2.500000e+05    107
1.500000e+05     82
5.000000e+05     77
               ... 
8.550000e+05      1
2.571190e+05      1
4.560000e+05      1
2.403000e+05      1
1.099536e+09      1
Name: Down Payment, Length: 1398, dtype: int64
1000.0      57
2000.0      46
20000.0     38
2.0         38
30000.0     34
            ..
6200.0       1
300.0        1
600.0        1
660000.0     1
5950.0       1
Name: Down Payment, Length: 127, dtype: int64
1.0          83
400000.0     77
500000.0     72
1000000.0    63
300000.0     62
             ..
1896000.0     1
172000.0      1
2975000.0     1
838000.0      1
300650.0      1
Name: Down Payment, Length: 1233, dtype: int64


In [34]:
df_totla=pd.concat(df).drop_duplicates().reset_index(drop=True)
df_totla.to_csv("total.csv")

***Fill_Nulls***

In [24]:
for i ,v in df_vilas[df_vilas['Compound']=='Madinaty']['Amenities'].value_counts().iloc[[2]].items():
    items=i
items

'Balcony Built in Kitchen Appliances Central A/C & heating Covered Parking Maids Room Pets Allowed Private Garden Security Pool Electricity Meter Water Meter Natural Gas Landline Elevator'

In [25]:
# madanity=''
# for i, v in df_vilas.loc[(df_vilas['Compound']=='Madinaty') & (df_vilas['location']=='Madinaty, Cairo') & (df_vilas['Amenities']!='x')]['Amenities'].value_counts().items():
#     madanity=i
#     break
# madanity

In [26]:
# df_vilas.loc[(df_vilas['Amenities']=='x') & (df_vilas['location']=='Madinaty, Cairo'),'Compound']='Madinaty'
# df_vilas.loc[(df_vilas['Amenities']=='x') & (df_vilas['location']=='Madinaty, Cairo'),'Amenities']=madanity

***Feature Extraction***


In [27]:
import numpy

st=set()
[[st.add(z) for  z in i.split() ]for i in [x for x in  df_vilas['Amenities'] if type(x)!=float ]]
x=list(st)
sub=['A/C','in','Meter','Private','Central','&','Built','Natural','Allowed', 'Room',]
Amenities={i:0 for  i in [i for i in x if i not in sub]}
print(Amenities.keys())

def x(z):
    Amenities2={'Pets': 0, 'Maids': 0, 'heating': 0, 'Balcony': 0, 'Garden': 0, 'Landline': 0, 'Security': 0, 'Parking': 0, 'Water': 0, 'Appliances': 0, 'Pool': 0, 'Gas': 0, 'Electricity': 0, 'Covered': 0, 'Kitchen': 0, 'Elevator': 0}
    try:
        if type(z)!=float or type(z)!=numpy.float64 :
            z=z.split()
            for i in z:
                if i in Amenities2.keys():
                    Amenities2[i]=1
            return  pd.Series(list(Amenities2.values()))
        return pd.Series(list(Amenities2.values()))
    except Exception as e:
        print(e)
        print(z)
        return pd.Series(list(Amenities2.values()))
df_vilas[list(Amenities.keys())]=df_vilas['Amenities'].apply(x)

dict_keys(['Covered', 'Water', 'heating', 'Security', 'Gas', 'Garden', 'Pets', 'Parking', 'Pool', 'Appliances', 'Elevator', 'Balcony', 'Electricity', 'Landline', 'Kitchen', 'Maids'])
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no attribute 'split'
nan
'float' object has no att

In [33]:
df_vilas.iloc[0]

price                                                     7400000.0
location                                         Sheikh Zayed، Giza
date                                            2022-03-09 00:00:00
title             استغل الفرصه وامتلك اخر فيلا في ليك ويست بتسهي...
link                    https://www.olx.com.eg/en/ad/8-IDcHny4.html
viwers                                                           54
negotiable                                                        0
Bedrooms                                                          4
Bathrooms                                                         4
Area (m²)                                                     338.0
Type                                              Stand Alone Villa
Payment Option                                  Cash or Installment
Compound                                                  Lake West
Delivery Term                                          Core & Shell
Amenities                                       

In [28]:
df_vilas_rent.isnull().sum()

price                1
location             0
date                 0
title                0
link                 0
viwers               0
negotiable           0
Amenities         1255
Bedrooms            37
Bathrooms           37
Area (m²)           35
Furnished         1061
Type                15
Compound          2202
Down Payment      4335
Payment Option    5136
Delivery Date     5141
Delivery Term     5141
cat                  0
Level             5142
dtype: int64

In [29]:
df_apartmant

,price,location,date,title,link,viwers,negotiable,Bedrooms,Bathrooms,Area (m²),Level,Type,Payment Option,Amenities,Furnished,Delivery Date,Compound,Delivery Term,Down Payment,cat
0,2560000.0,Smoha، Alexandria,2022-01-01,شقة للبيع فى حى مروج 160م بمقدم 10% و اقساط عل...,https://www.olx.com.eg/en/ad/160-10-5-IDbV1PO....,67,0,3,2,160.0,7.0,Apartment,Installment,NaN,NaN,NaN,NaN,NaN,NaN,Sell
1,1650000.0,Nasr City، Cairo,2022-03-10,للبيع شقه روعه ١٦٠م مدينه نصر,https://www.olx.com.eg/en/ad/-IDcHAPd.html,629,0,3,2,160.0,1.0,Apartment,Cash,Balcony Built in Kitchen Appliances Private Ga...,No,Ready to move,NaN,NaN,NaN,Sell
2,2380000.0,"New Capital City, Cairo",2022-02-13,امتلك افضل شقة بالعاصمة تشطيب كامل بالمطبخ,https://www.olx.com.eg/en/ad/-IDcDVHS.html,28,1,2,2,119.0,0.0,Apartment,Cash or Installment,Balcony Covered Parking Maids Room Pets Allowe...,NaN,Ready to move,Atika,Finished,NaN,Sell
3,665000.0,New Cairo - El Tagamoa، Cairo,2022-03-07,شقه 130م بجاردن بكمبوند البروج فالشروق,https://www.olx.com.eg/en/ad/130-IDcH5kM.html,360,0,2,2,130.0,3.0,Apartment,Cash or Installment,Balcony Private Garden Security Covered Parkin...,NaN,NaN,NaN,NaN,NaN,Sell
4,3900000.0,Madinaty، Cairo,2022-03-10,بسعر لقطه ب مدينتى شقه للبيع نموذج 700 بافضل م...,https://www.olx.com.eg/en/ad/700-320-IDcHA2D.html,238,0,4,4,320.0,5.0,Apartment,Cash,NaN,No,Ready to move,Madinaty,Finished,NaN,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20312,1750000.0,New Cairo - El Tagamoa، Cairo,2022-03-05,شقة ١٩٠ م ب درة القاهرة أمام كمبوند اكاسيا أما...,https://www.olx.com.eg/en/ad/-IDcgTes.html,292,1,3,3,190.0,1.0,Apartment,NaN,NaN,No,NaN,NaN,NaN,NaN,Sell
20313,900000.0,Zarqa، Damietta,2022-03-05,شقة للبيع,https://www.olx.com.eg/en/ad/-IDcCi0O.html,77,1,3,2,165.0,1.0,Apartment,Installment,Elevator,No,NaN,NaN,NaN,30000.0,Sell
20314,1850000.0,Heliopolis، Cairo,2022-03-05,لراغبي الاستثمار بالتقسيط ارضي مدخل خاص 200م ن...,https://www.olx.com.eg/en/ad/200-IDcGO9M.html,237,0,4,2,200.0,0.0,Apartment,Cash or Installment,Security Electricity Meter Water Meter,No,Ready to move,NaN,Semi Finished,NaN,Sell
20315,1980000.0,New Cairo - El Tagamoa، Cairo,2022-03-05,شقه للبيع في كمبوند جالريا مون فالي,https://www.olx.com.eg/en/ad/-IDcGO8w.html,111,0,2,2,125.0,2.0,Apartment,Cash or Installment,Balcony Covered Parking Pets Allowed Security ...,No,Ready to move,Galleria Moon Valley,Semi Finished,316000.0,Sell


In [30]:
df_apertment_rent.isnull().sum()

price                 0
location              0
date                  0
title                 0
link                  0
viwers                0
negotiable            0
Bedrooms              0
Bathrooms             0
Area (m²)             0
Furnished          4687
Level                 1
Type                  4
Compound          15374
Amenities          7148
Down Payment      16927
cat                   0
Payment Option    21735
Delivery Date     21735
Delivery Term     21735
dtype: int64

In [31]:
df_state21.isnull().sum()

link               0
date               1
title              1
viwers             0
location           1
Price              1
sign               1
Amenities       2227
Bedrooms           1
Bathrooms          1
Area (m²)          1
Furnished       1444
Level              1
Type               1
Compound        4841
Down Payment    4928
table           6518
dtype: int64